In [1]:
%load_ext autoreload
%autoreload 2

In [97]:
import pandas as pd 
import numpy as np
import requests 
from bs4 import BeautifulSoup 
import re
from clean_data import *
from webscrap import *

import copy
#import dtale
from datetime import datetime
import altair as alt
import theme_desarroio

In [3]:
import dtale

In [4]:
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)

### Pendiente: 
1) funcion que clasifica campo periodo, en fecha. 
2) limpiar periodos unidos. 
3) clasificar sexo de ministrxs

In [5]:
url_dic = {
    'pcm': 'https://es.wikipedia.org/wiki/Anexo:Presidentes_del_Consejo_de_Ministros_del_Per%C3%BA',
    'rree': 'https://es.wikipedia.org/wiki/Ministro_de_Relaciones_Exteriores_del_Per%C3%BA',
    'defensa': 'https://es.wikipedia.org/wiki/Ministro_de_Defensa_del_Per%C3%BA', # necesita limpiar encabezados
    'economia': 'https://es.wikipedia.org/wiki/Ministro_de_Econom%C3%ADa_y_Finanzas_del_Per%C3%BA',## necesita limpiar fechas
    'interior': 'https://es.wikipedia.org/wiki/Ministro_del_Interior_del_Per%C3%BA', ## necesita limpiar fechas
    'justicia': 'https://es.wikipedia.org/wiki/Ministro_de_Justicia_y_Derechos_Humanos_del_Per%C3%BA',
    'educacion': 'https://es.wikipedia.org/wiki/Ministro_de_Educaci%C3%B3n_del_Per%C3%BA',
    'salud': 'https://es.wikipedia.org/wiki/Ministro_de_Salud_del_Per%C3%BA',
    'agro': 'https://es.wikipedia.org/wiki/Ministro_de_Desarrollo_Agrario_y_Riego_del_Per%C3%BA',
    'trabajo': 'https://es.wikipedia.org/wiki/Ministro_de_Trabajo_y_Promoci%C3%B3n_del_Empleo_del_Per%C3%BA',
    'produccion': 'https://es.wikipedia.org/wiki/Ministro_de_la_Producci%C3%B3n_del_Per%C3%BA', # solo produccion
    'comercio': 'https://es.wikipedia.org/wiki/Ministro_de_Comercio_Exterior_y_Turismo_del_Per%C3%BA', 
    'energia': 'https://es.wikipedia.org/wiki/Ministro_de_Energ%C3%ADa_y_Minas_del_Per%C3%BA', 
    'transporte': 'https://es.wikipedia.org/wiki/Anexo:Ministros_de_Transportes_y_Comunicaciones_del_Per%C3%BA',
    'vivienda': 'https://es.wikipedia.org/wiki/Ministro_de_Vivienda,_Construcci%C3%B3n_y_Saneamiento_del_Per%C3%BA',
    'mujer': 'https://es.wikipedia.org/wiki/Ministerio_de_la_Mujer_y_Poblaciones_Vulnerables',
    'ambiente': 'https://es.wikipedia.org/wiki/Ministro_del_Ambiente_del_Per%C3%BA',
    'cultura': 'https://es.wikipedia.org/wiki/Ministro_de_Cultura_del_Per%C3%BA',
    'desarrollo': 'https://es.wikipedia.org/wiki/Ministro_de_Desarrollo_e_Inclusi%C3%B3n_Social_del_Per%C3%BA'
    
}

# nos dice cuantos 
url_ntable = {
    'pcm': 1,
    'rree': 1,
    'defensa': 1, # necesita limpiar encabezados
    'economia': 1,## necesita limpiar fechas
    'interior': 1, ## necesita limpiar fechas
    'justicia': 2, ## debe ser 2
    'educacion': 1,
    'salud': 1,
    'agro': 1,
    'trabajo': 2, ## debe ser 2, desde 2003
    'produccion': 2, ## desde 2002
    'comercio': 2, # desde el 2003, falta completar anios
    'energia': 1,
    'transporte': 1,
    'vivienda': 2, #desde el 2002, tiene que ser mastranst
    'mujer': 2 ,# homogenizar campos, completar tiene q ser 2
    'ambiente': 1,
    'cultura': 1,
    'desarrollo': 1
}

In [6]:
#df = get_df_from_soup(aa)
scrap = {}
for key, val in url_dic.items():
    scrap[key] = get_wiki_table(val, url_ntable[key])

3
6
1
2
5
13
2
1
1
5
2
3
1
1
2
3
1
1
1


In [73]:
d2 = copy.deepcopy(scrap)

In [74]:
#agregando una variable necesaria
d2['vivienda']['Gobierno'] = ''        


for key, val in d2.items():
    d2[key] = limpia_base(val)
    
#agregando fin necesario
d2['educacion']['fin'] = d2['educacion']['inicio'].shift(-1)
d2['educacion'] = d2['educacion'].iloc[d2['educacion'].index > 48]
d2['educacion'] = d2['educacion'].reset_index(drop = True)

periodos separados: pcm, rree, defensa, salud, vivienda, cultura  
periodos unidos: economia, interior, justicia, agro, trabajo, produccion, comercio, energia, transporte, mujer, ambiente, desarrollo  
sin fin, solo inicio: educacion

In [75]:
por_separar = ['economia', 'interior', 'justicia','agro', 'trabajo',
'produccion', 'comercio', 'energia', 'transporte', 'mujer', 'ambiente', 'desarrollo']



for key in d2.keys():
    if key in por_separar:
        #print(key)
        d2[key].dropna(subset=['Ministro'], inplace = True)
        d2[key]['Periodo'] = d2[key]['Periodo'].apply(lambda x: clean_dash(x))
        d2[key]['Periodo_sp'] = d2[key]['Periodo'].str.split('-')
        d2[key]['inicio'] = d2[key]['Periodo_sp'].apply(lambda x: x[0])
        d2[key]['fin'] = d2[key]['Periodo_sp'].apply(lambda x: x[1])
        

In [76]:
for key in d2.keys():
    print(key)
    d2[key] = d2[key][['Ministro', 'Presidente', 'inicio', 'fin']]
    d2[key].loc[len(d2[key]) - 1, "fin"] = "30 de julio de 2021"

pcm
rree
defensa
economia
interior
justicia
educacion
salud
agro
trabajo
produccion
comercio
energia
transporte
vivienda
mujer
ambiente
cultura
desarrollo


/Users/ccsuehara/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-76-42057123f44d>:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [77]:
for key in d2.keys():
    print(key)
    d2[key] = get_fechas(d2[key], 'inicio')
    d2[key] = get_fechas(d2[key], 'fin')

pcm
rree
defensa
economia
interior
justicia
educacion
salud
agro
trabajo
produccion
comercio
energia
transporte
vivienda
mujer
ambiente
cultura
desarrollo


/Users/ccsuehara/GH folders/data_vizzes/genero_gabinete/scripts/clean_data.py:101: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [78]:
nombres_hombres = set_nombres('../data/hombres.csv')
nombres_mujeres = set_nombres('../data/mujeres.csv')

new_set = {'Ana', 'Beatriz', 'Carmen', 'Claudia',
           'Elizabeth', 'Maria', 'Mercedes', 'Patricia', 'Pilar', 'Silvia', 'Teresa', 'Rosario'}

nombres_hombres = nombres_hombres - new_set

In [79]:
nombres_mujeres.intersection(nombres_hombres)

{'Abad',
 'Abel',
 'Abigail',
 'Adalid',
 'Adrian',
 'Adriana',
 'Alejandr',
 'Alexi',
 'Alexis',
 'Ali',
 'Andres',
 'Antonia',
 'Antonio',
 'Apolinar',
 'Areli',
 'Arturo',
 'Ascencion',
 'Asuncion',
 'Bernabe',
 'Buenaventura',
 'Carlos',
 'Christian',
 'Cleofas',
 'Concepcion',
 'Consuelo',
 'Cristhian',
 'Cristian',
 'Cruz',
 'David',
 'De',
 'Dennis',
 'Dolores',
 'Dorian',
 'Eden',
 'Edgar',
 'Edi',
 'Edith',
 'Eduviges',
 'Eleazar',
 'Eli',
 'Elias',
 'Eliazar',
 'Eliud',
 'Eliut',
 'Eliuth',
 'Eloy',
 'Elvis',
 'Emir',
 'Encarnacion',
 'Eric',
 'Ever',
 'Felix',
 'Floricel',
 'Francis',
 'Francisc',
 'Francisco',
 'Gabriel',
 'Gabriela',
 'Gerardo',
 'Guadalupe',
 'Guillermo',
 'Heber',
 'Hernandez',
 'Ines',
 'Inocente',
 'Irais',
 'Iran',
 'Isabel',
 'Isaias',
 'J',
 'Jared',
 'Javier',
 'Jesus',
 'Joel',
 'Jorge',
 'Josafat',
 'Jose',
 'Josefina',
 'Juan',
 'Juana',
 'Leonides',
 'Leonor',
 'Leticia',
 'Lidia',
 'Lilia',
 'Lopez',
 'Loreto',
 'Luz',
 'Ma',
 'Maciel',
 'Magd

In [80]:
for key in d2.keys():
    d2[key]['nombre'] = d2[key]['Ministro'].str.split(' ')
    d2[key]['gabinete'] = key
    d2[key]['nombre'] = d2[key]['nombre'].apply(lambda x: x[0].replace("é", "e").replace("á", "a").replace("í", "i").replace("ó", "o").replace("ú", "u"))
    d2[key]['genero'] = d2[key]['nombre'].apply(lambda x: pertenece_nombres(x,nombres_hombres, nombres_mujeres))

In [81]:
ministros = pd.DataFrame()
for key, val in d2.items():
    ministros = pd.concat([ministros,d2[key]])

In [82]:
ministros = ministros.reset_index(drop = True)

In [83]:
adicional_h = ['Óscar', 'Álvaro', 'Edmer', 'Rudecindo', 'Ludwig', 'Ántero', 'Orestes', 'Cluber', 'Piero', 'Édgar',
              'Baldo', 'Glodomiro', 'Mirko', 'Joffre', 'Yehude', 'Elivio', 'Jakke', 'Guido', 'Idel',
              'Gino', 'Nils', 'Kurt']
adicional_m = ['Ángela', 'Eda', 'Fiorella', 'Midori', 'Ulla', 'Kirla', 'Franca', 'Ariela', 'Lieneke']

In [84]:
ministros['genero_2'] = ministros['nombre'].apply(lambda x: pertenece_nombres(x,adicional_h, adicional_m))



In [85]:
ministros['genero'] = np.where(ministros['genero'] == 'no match', ministros['genero_2'], ministros['genero'])
                      

In [86]:
ministros.drop(columns = ['genero_2'], inplace = True)

In [87]:
ministros["start"] = pd.to_datetime(ministros['f_inicio'])
ministros["end"] = pd.to_datetime(ministros['f_fin'])

ministros['year_end'] = ministros["end"].dt.year

In [209]:
mini = ministros.loc[ministros.year_end > 1999]
mini = mini.copy(deep = True)
mini = mini.reset_index(drop = True)

In [210]:

mini.replace({'Presidente': {'Pedro Pablo Kuczynski': 'PPK',
                            'Alan García Pérez': 'Alan García', 
                           'Martín Vizcarra Cornejo': 'Martín Vizcarra',
                           'Alejandro Toledo Manrique': 'Alejandro Toledo',
                           'Ollanta Humala Tasso': 'Ollanta Humala',
                           'Alan García(2.º gob.)': 'Alan García',
                           'Alan García Pérez (2.º gob.)': 'Alan García',
                           'Valentín Paniagua Corazao': 'Valentín Paniagua',
                           'Francisco Sagasti Hochhausler': 'Francisco Sagasti',
                           'Manuel Merino de Lama':'Manuel Merino',
                           'Valentín Paniagua (transitorio)': 'Valentín Paniagua',
                           'Alberto Fujimori Fujimori': 'Alberto Fujimori',
                           'Alan García (2.º gobierno)': 'Alan García',
                           'Valentín Paniagua (Presidente transitorio)' : 'Valentín Paniagua',
                           'Alan García (2º gob.)': 'Alan García'}}, inplace = True)


mini.replace({'gabinete': {'mujer':'Mujer', 'desarrollo':'Desarrollo',
                           'comercio':'Comercio', 'ambiente':'Ambiente',
                            'cultura':'Cultura', 'salud':'Salud', 'justicia':'Justicia',
                           'trabajo':'Trabajo', 'educacion':'Educación',
                            'produccion':'Producción', 'rree':'RREE', 
                           'economia':'Economía', 'pcm':'PCM', 'interior':'Interior',
                           'transporte':'Transporte','vivienda':'Vivienda',
                           'energia':'Energía', 'agro':'Agricultura', 'defensa':'Defensa'}}, inplace = True)

In [211]:
mini.Presidente.value_counts()


Martín Vizcarra      86
Ollanta Humala       74
Alejandro Toledo     73
Alan García          67
PPK                  44
Francisco Sagasti    23
Alberto Fujimori     21
Manuel Merino        19
Valentín Paniagua    15
Pedro Castillo        1
Name: Presidente, dtype: int64

In [212]:
mini = mini.loc[mini.Presidente != 'Alberto Fujimori']
mini = mini.copy(deep = True)

In [213]:
ss = mini.loc[mini.groupby('Presidente').start.idxmin()]
ss = ss[['Presidente', 'start']]
ss.columns = ['Presidente', 'start_gob']
mini = pd.merge(mini, ss, on = ['Presidente'], how = 'left' )

In [214]:
mini['start_fake'] =  mini['start'] - mini['start_gob']
mini['end_fake'] = mini['end'] - mini['start_gob']
mini['start_fake'] = mini['start_fake'].dt.days
mini['end_fake'] = mini['end_fake'].dt.days


In [242]:
extra = pd.read_csv('../data/nuevo_gabinete.csv')

In [233]:
extra.loc[extra['gabinete'] == 'Economía', 'genero'] = np.nan
extra.loc[extra['gabinete'] == 'Justicia', 'genero'] = np.nan

In [243]:
extra

,gabinete,Presidente,genero
0,Interior,Pedro Castillo,Hombre
1,PCM,Pedro Castillo,Hombre
2,Producción,Pedro Castillo,Hombre
3,Trabajo,Pedro Castillo,Hombre
4,Agricultura,Pedro Castillo,Hombre
5,Defensa,Pedro Castillo,Hombre
6,Energía,Pedro Castillo,Hombre
7,Salud,Pedro Castillo,Hombre
8,Mujer,Pedro Castillo,Mujer
9,RREE,Pedro Castillo,Hombre


In [244]:
order = ['Mujer', 'Desarrollo', 'Comercio', 'Ambiente',
'Cultura', 'Salud', 'Justicia', 'Trabajo', 'Educación',
'Producción', 'RREE', 'Transporte','Energía','Economía', 'PCM',  
'Vivienda', 'Interior', 'Agricultura', 'Defensa']


In [245]:
castillo = alt.Chart(extra).mark_circle( size = 600).encode(
   y=alt.Y('gabinete', 
            sort = order, title = None, axis = None),
    color=alt.Color('genero')
)

In [246]:
rad = 10

base = alt.Chart(mini, width = 1000, height = 600).mark_bar(cornerRadiusTopLeft= rad,
                                             cornerRadiusTopRight = rad,
                                             cornerRadiusBottomLeft = rad, 
                                             cornerRadiusBottomRight = rad, ).encode(
    x= alt.X('start', title = 'Período por cada ministra/o', axis=alt.Axis(ticks=False, domain=False)),
    x2= 'end',
    y=alt.Y('gabinete', 
            sort = order, title = None, axis=alt.Axis(ticks=False, domain=False, orient='right')), 
    color=alt.Color('genero')).properties(title={
        "text": ["Mujeres como ministras de Estado 2001-2021"], 
        "subtitle": ["Fuente: Wikipedia",
                    "Elaboración: desarro.io"],
        "subtitleFont": "Sanchez",
    })

#     ,
#     column='Presidente'
# ).resolve_scale(
#     x="independent").configure_facet(
#     spacing=1
# )




In [247]:
theme = theme_desarroio.theme_desarroio()
final = alt.hconcat(base , castillo).configure_axis(
    labelFontSize=15,
    titleFontSize=15).resolve_scale(
    y='shared'
)
final

alt.HConcatChart(...)

In [248]:
# rad = 10

# base2 = alt.Chart(mini, width = 1000, height = 600).mark_bar(cornerRadiusTopLeft= rad,
#                                              cornerRadiusTopRight = rad,
#                                              cornerRadiusBottomLeft = rad, 
#                                              cornerRadiusBottomRight = rad).encode(
#     x= alt.X('start', title = 'Período por cada ministra/o', axis=alt.Axis(ticks=False, domain=False)),
#     x2= 'end',
#     y=alt.Y('gabinete', 
#             sort = order, title = None, axis = None), 
#     color=alt.Color('genero')).properties(title={
#         "text": ["Mujeres como ministras de Estado 2001-2021"], 
#         "subtitle": ["Fuente: Wikipedia",
#                     "Elaboración: desarro.io"],
#         "subtitleFont": "Sanchez",
#     })

# #     ,
# #     column='Presidente'
# # ).resolve_scale(
# #     x="independent").configure_facet(
# #     spacing=1
# # )




In [199]:
# castillo2 = alt.Chart(extra).mark_circle( size = 170).encode(
#    y=alt.Y('gabinete', 
#             sort = order, title = None,  axis=alt.Axis(ticks=False, domain=False, orient='right')),
#     color=alt.Color('genero')
# )

In [202]:
# theme = theme_desarroio.theme_desarroio()
# final2 = alt.hconcat(base2 , castillo2).configure_axis(
#     labelFontSize=15,
#     titleFontSize=15).resolve_scale(
#     y='shared'
# )
# final2

alt.HConcatChart(...)

In [ ]:
cars